# An example workflow of the QAOA package

This notebook walks through a simple example workflow, from start to finish.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from qaoa.cost_function import QAOACostFunctionOnQVM, QAOACostFunctionOnWFSim
from pyquil.api import local_qvm, WavefunctionSimulator
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters that we want to demo
from qaoa.parameters import AdiabaticTimestepsQAOAParameters,\
AlternatingOperatorsQAOAParameters, AbstractQAOAParameters, GeneralQAOAParameters,\
QAOAParameterIterator, FourierQAOAParameters

from vqe.optimizer import scipy_optimizer

## Set up the problem hyperparameters

We start by setting up the Hamiltonian encoding the cost function we wish to minimise, as well as the timesteps to run QAOA. We do so by first using the `AbstractQAOAParameterClass`, however we could just as easily set up any specific parametrisation directly - see the notebook _"A demo how to work with `QAOAParameter` classes"_ for more details. [Should hyperlink to that notebook when online]
 

In [2]:
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

timesteps = 2
myAbstractParams = AbstractQAOAParameters([hamiltonian,timesteps])
print(myAbstractParams)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	single_qubit_coeffs: [-0.5]
	qubits_pairs: [[0, 1], [0, 2]]
	pair_qubit_coeffs: [0.7 1.2]
	timesteps: 2



## Specfify the variable parameters and create corresponding `QAOAParameters` object

Here we will set up `AlternatingOperatorsQAOAParameters` - this corresponds to the parametrisation of the original QAOA paper by Farhi et al, where the mixer and cost Hamiltonian each have one angle per timestep (giving a total of 2p parameters to optimise over).

In [3]:
# Specify some angles
betas          = [0.1, 0.6]
gammas_singles = [0.4, 0.5]
gammas_pairs   = [0.1, 0.3]
parameters = (betas, gammas_singles, gammas_pairs)

myAlternatingParams = AlternatingOperatorsQAOAParameters.from_AbstractParameters(myAbstractParams,parameters)
print(myAlternatingParams)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.1 0.6]
	gammas_singles: [0.4 0.5]
	gammas_pairs: [0.1 0.3]



## Set up the cost function and running the optimiser

Let's work on the wavefunction simulator, which we have imported (`local_qvm`). We create the `cost_function` object, with all the necessary inputs [Make another demo of the `cost_functions` or expand the details here]. 

[NOTES: (1) look into the `noisy` attribute here - has it been properly tested and benchmarked? (2) demo the use of the `log` attribute]

In [4]:
# Set up the WavefunctionSimulator
sim = WavefunctionSimulator()

In [9]:
with local_qvm():
    cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                            params=myAlternatingParams,
                                            sim=sim,
                                            return_standard_deviation=True,
                                            noisy=False,
                                            log=[])
    res = scipy_optimizer(cost_function, myAlternatingParams.raw(), epsilon=1e-3,
                          maxiter=180)
res


     fun: -1.8002540783951095
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 74
  status: 1
 success: True
       x: array([ 1.03262024, -0.30468633,  0.17348849,  0.00932234, -0.51050857,
       -0.91574438])

[NOTE (EWAN): I seem to be having issues running the above line of code - if I choose to high a value of `maxiter`, the thing does not seem to terminate.]

We can check this against the value we should get, by finding the lowest eigenvalue of `hamiltonian`.

In [10]:
ham_matrix = hamiltonian.matrix()
eigs = np.linalg.eigvals(ham_matrix)
min(eigs)

(-2.4+0j)

Clearly we have not found the ground state energy. We can try increasing the number of timesteps:

In [3]:
betas          = [0.1, 0.6, 0.8]
gammas_singles = [0.4, 0.5, 0.6]
gammas_pairs   = [0.1, 0.3, 0.5]
parameters = (betas, gammas_singles, gammas_pairs)

myAlternatingParams_p3 = AlternatingOperatorsQAOAParameters([hamiltonian,3],parameters)

In [4]:
sim = WavefunctionSimulator()
with local_qvm():
    cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                            params=myAlternatingParams_p3,
                                            sim=sim,
                                            return_standard_deviation=True,
                                            noisy=False,
                                            log=[])
    res = scipy_optimizer(cost_function, myAlternatingParams_p3.raw(), epsilon=1e-3,
                          maxiter=100)
res

     fun: -1.856224838493715
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 100
  status: 2
 success: False
       x: array([-0.67192842,  0.4897888 ,  1.87441185,  0.50840428,  0.1144147 ,
        0.19984343, -0.10746511,  0.47006071,  0.95271429])

[Again the runtime issue seems to be preventing me from going much beyond 100 iterations, but we see that the objective function is at least getting smaller].

Since we don't seem to be doing very well with `AlternatingOperatorsQAOAParameters`, let's try the `GeneralQAOAParameters` parametrisation instead.

In [6]:
myGeneralParams_p3 = GeneralQAOAParameters.linear_ramp_from_hamiltonian(hamiltonian,timesteps=3)

with local_qvm():
    cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                            params=myGeneralParams_p3,
                                            sim=sim,
                                            return_standard_deviation=True,
                                            noisy=False,
                                            log=[])
    res = scipy_optimizer(cost_function, myGeneralParams_p3.raw(), epsilon=1e-3,
                          maxiter=100)
res

     fun: -2.3399445785158957
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 100
  status: 2
 success: False
       x: array([ 0.77101167,  0.97373301,  0.1589382 ,  0.04377861,  0.48096282,
        0.83226812,  0.00692921,  0.22352015,  0.31611752,  1.18235741,
        0.82862582,  1.1058874 ,  0.05529414, -0.02216295,  1.18547021,
        0.39355261,  0.06226389,  0.64904335])

Bingo - almost there!